In [235]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from random import choice
import random 
from math import isnan

In [422]:
### Load Table Up
### Dropping Division due to not properly labeling each state by their respective division 
restaurant_df = pd.read_pickle("restaurant_df_pickle.pkl")
restaurant_df.drop(["Division"], axis=1, inplace=True )
restaurant_df.head()

,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Menu Item,Menu Item Price
0,534f07e9498e5cc70137182b,The Causeway Restaurant and Pub,42.364659,-71.062912,MA,Boston,BBQ Joint,7.0,55,Moderate,[],[]
1,593d9fb5c876c8327eef128d,Rusty Can,42.755437,-70.938839,MA,Byfield,BBQ Joint,8.0,14,Moderate,[],[]
2,4ba2b7caf964a520211338e3,Joff's Backyard Grill,42.084574,-71.471883,MA,Bellingham,BBQ Joint,7.9,18,Moderate,[],[]
3,5c5efed8419a9e002ce8ea9c,The Smoke Shop BBQ - Assembly Row,42.392249,-71.078180,MA,Somerville,Restaurant,-1.0,0,Moderate,"[17Th Street Soul Rolls, Hot Links & Pimento C...","[9.00, 7.50, 9.00, 10.00, 12.00, 9.00, 9.00, 8..."
4,5ca25d74dd12f8002c74364b,Flip The Bird,42.559270,-70.881620,MA,Beverly,BBQ Joint,-1.0,0,Moderate,[],[]


In [256]:
### Load in State Capital Table 
state_capital = pd.read_pickle("state_div.pkl")
state_capital.head()

,State,Capital,Division
0,MA,Boston,1
1,CT,Hartford,1
2,RI,Providence,1
3,NJ,Trenton,2
4,NY,Albany,2


In [257]:
### Clean Up Method 

### Clean up 1: Obtain length of menu list and average of price for each restaurant 
len_menu_list = []
avg_price_list = []

for i in range(restaurant_df.shape[0]) : 
    len_menu_list.append(len(restaurant_df.iloc[i]["Menu Item"]) )
    
    price_list = restaurant_df.iloc[i]["Menu Item Price"]
    
    if price_list != [] : 
        if type(price_list[0]) == dict : 
            price_list = [ key['0'] if '0' in key and 'Add' not in key['0'] and 'Whole' not in key['0']  else '0' for key in price_list ]

    
    if "Unknown" in price_list or price_list == [] : 
        avg_price_list.append(np.nan)
    else : 
        avg_price_list.append( np.average(list(map(float, price_list ) ) ) )  

restaurant_df.drop(columns=["Menu Item", "Menu Item Price"], inplace=True)
restaurant_df["Average Menu Length"] = len_menu_list
restaurant_df["Average Item Price"] = avg_price_list

restaurant_df.head() 

,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Average Menu Length,Average Item Price
0,534f07e9498e5cc70137182b,The Causeway Restaurant and Pub,42.364659,-71.062912,MA,Boston,BBQ Joint,7.0,55,Moderate,0,NaN
1,593d9fb5c876c8327eef128d,Rusty Can,42.755437,-70.938839,MA,Byfield,BBQ Joint,8.0,14,Moderate,0,NaN
2,4ba2b7caf964a520211338e3,Joff's Backyard Grill,42.084574,-71.471883,MA,Bellingham,BBQ Joint,7.9,18,Moderate,0,NaN
3,5c5efed8419a9e002ce8ea9c,The Smoke Shop BBQ - Assembly Row,42.392249,-71.078180,MA,Somerville,Restaurant,-1.0,0,Moderate,59,12.457627
4,5ca25d74dd12f8002c74364b,Flip The Bird,42.559270,-70.881620,MA,Beverly,BBQ Joint,-1.0,0,Moderate,0,NaN


In [258]:
### Clean up 2: Replace all invalid values with NaN and Create a Division Column 
div_list = [] 

for i in range(restaurant_df.shape[0]) : 
    div_num = state_capital[state_capital["State"] == restaurant_df.iloc[i]["State"] ]["Division"].values[0] 
    div_list.append(div_num)

restaurant_df["Division"] = div_list
restaurant_df["Average Menu Length"].replace(0, np.nan, inplace=True)
restaurant_df["Average Item Price"].replace(0, np.nan, inplace=True)
restaurant_df["Rating"].replace(-1.0, np.nan, inplace=True)
restaurant_df["Number of Rating"].replace(0, np.nan, inplace=True)

price_encode = {"Price Tier":     {"Cheap": 0, "Moderate": 1, "Expensive": 2, "Very Expensive": 3 , "Not Available": np.nan} }
restaurant_df.replace(price_encode, inplace=True)

restaurant_df.head() 


,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Average Menu Length,Average Item Price,Division
0,534f07e9498e5cc70137182b,The Causeway Restaurant and Pub,42.364659,-71.062912,MA,Boston,BBQ Joint,7.0,55.0,1.0,NaN,NaN,1
1,593d9fb5c876c8327eef128d,Rusty Can,42.755437,-70.938839,MA,Byfield,BBQ Joint,8.0,14.0,1.0,NaN,NaN,1
2,4ba2b7caf964a520211338e3,Joff's Backyard Grill,42.084574,-71.471883,MA,Bellingham,BBQ Joint,7.9,18.0,1.0,NaN,NaN,1
3,5c5efed8419a9e002ce8ea9c,The Smoke Shop BBQ - Assembly Row,42.392249,-71.078180,MA,Somerville,Restaurant,NaN,NaN,1.0,59.0,12.457627,1
4,5ca25d74dd12f8002c74364b,Flip The Bird,42.559270,-70.881620,MA,Beverly,BBQ Joint,NaN,NaN,1.0,NaN,NaN,1


In [259]:
### Save Cleaned Table to pickle file and test if file was saved properly 
restaurant_df.to_pickle("restaurant_clean_df_pickle.pkl")
restaurant_df = pd.read_pickle("restaurant_clean_df_pickle.pkl")
restaurant_df.head()

,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Average Menu Length,Average Item Price,Division
0,534f07e9498e5cc70137182b,The Causeway Restaurant and Pub,42.364659,-71.062912,MA,Boston,BBQ Joint,7.0,55.0,1.0,NaN,NaN,1
1,593d9fb5c876c8327eef128d,Rusty Can,42.755437,-70.938839,MA,Byfield,BBQ Joint,8.0,14.0,1.0,NaN,NaN,1
2,4ba2b7caf964a520211338e3,Joff's Backyard Grill,42.084574,-71.471883,MA,Bellingham,BBQ Joint,7.9,18.0,1.0,NaN,NaN,1
3,5c5efed8419a9e002ce8ea9c,The Smoke Shop BBQ - Assembly Row,42.392249,-71.078180,MA,Somerville,Restaurant,NaN,NaN,1.0,59.0,12.457627,1
4,5ca25d74dd12f8002c74364b,Flip The Bird,42.559270,-70.881620,MA,Beverly,BBQ Joint,NaN,NaN,1.0,NaN,NaN,1


In [449]:
restaurant_df = pd.read_pickle("restaurant_clean_df_pickle.pkl")
restaurant_df.head()

,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Average Menu Length,Average Item Price,Division
0,534f07e9498e5cc70137182b,The Causeway Restaurant and Pub,42.364659,-71.062912,MA,Boston,BBQ Joint,7.0,55.0,1.0,NaN,NaN,1
1,593d9fb5c876c8327eef128d,Rusty Can,42.755437,-70.938839,MA,Byfield,BBQ Joint,8.0,14.0,1.0,NaN,NaN,1
2,4ba2b7caf964a520211338e3,Joff's Backyard Grill,42.084574,-71.471883,MA,Bellingham,BBQ Joint,7.9,18.0,1.0,NaN,NaN,1
3,5c5efed8419a9e002ce8ea9c,The Smoke Shop BBQ - Assembly Row,42.392249,-71.078180,MA,Somerville,Restaurant,NaN,NaN,1.0,59.0,12.457627,1
4,5ca25d74dd12f8002c74364b,Flip The Bird,42.559270,-70.881620,MA,Beverly,BBQ Joint,NaN,NaN,1.0,NaN,NaN,1


In [450]:
def remove_outlier( df, outlier_name, remove_amount, remove_confirm) : 
    x = df[outlier_name].values
    x = x[~np.isnan(x)]

    outliers = np.flip(np.sort(x))[0:remove_amount]
    
    ### Remove Outlier Prices 
    if remove_confirm : 
        return df[ ~df[outlier_name].isin(outliers) ]
    else :
        return outliers 

    
    

In [457]:
### Removing any significant Outliers 
restaurant_df_outlier_rm = remove_outlier(restaurant_df, "Number of Rating", 20, 1)
restaurant_df_outlier_rm = remove_outlier(restaurant_df_outlier_rm, "Average Menu Length", 60, 1)
restaurant_df_outlier_rm = remove_outlier( restaurant_df_outlier_rm, "Average Item Price", 7, 1)
restaurant_df_outlier_rm.reset_index(drop=True, inplace=True )

### Obtain array of valid values for each column 
ratings = np.array(restaurant_df_outlier_rm[ ~np.isnan(restaurant_df_outlier_rm["Rating"]) ]["Rating"]) 
num_rating = np.array(restaurant_df_outlier_rm[ ~np.isnan(restaurant_df_outlier_rm["Number of Rating"]) ]["Number of Rating"]) 
avg_menu = np.array(restaurant_df_outlier_rm[ ~np.isnan(restaurant_df_outlier_rm["Average Menu Length"]) ]["Average Menu Length"]) 
avg_item = np.array(restaurant_df_outlier_rm[ ~np.isnan(restaurant_df_outlier_rm["Average Item Price"]) ]["Average Item Price"]) 

random.seed(42)

In [458]:
### Replacing each respective NaN with a random value from its column 

for i in range(restaurant_df_outlier_rm.shape[0]) : 
    row = restaurant_df_outlier_rm.iloc[i]
        
    if isnan(row["Rating"]) : 
        restaurant_df_outlier_rm.at[i, "Rating"] = choice(ratings) 
    
    if isnan(row["Number of Rating"])  : 
        restaurant_df_outlier_rm.at[i, "Number of Rating"] = choice(num_rating) 
    
    if isnan(row["Price Tier"]) :
        restaurant_df_outlier_rm.at[i, "Price Tier"] = 1 

    if isnan(row["Average Menu Length"] ): 
        restaurant_df_outlier_rm.at[i, "Average Menu Length"] = choice(avg_menu) 
        
    if isnan(row["Average Item Price"]) : 
        restaurant_df_outlier_rm.at[i, "Average Item Price"] = choice(avg_item)


In [459]:
restaurant_df_outlier_rm.head()

,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Average Menu Length,Average Item Price,Division
0,534f07e9498e5cc70137182b,The Causeway Restaurant and Pub,42.364659,-71.062912,MA,Boston,BBQ Joint,7.0,55.0,1.0,45.0,9.595556,1
1,593d9fb5c876c8327eef128d,Rusty Can,42.755437,-70.938839,MA,Byfield,BBQ Joint,8.0,14.0,1.0,18.0,8.618889,1
2,4ba2b7caf964a520211338e3,Joff's Backyard Grill,42.084574,-71.471883,MA,Bellingham,BBQ Joint,7.9,18.0,1.0,27.0,8.490000,1
3,5c5efed8419a9e002ce8ea9c,The Smoke Shop BBQ - Assembly Row,42.392249,-71.078180,MA,Somerville,Restaurant,7.7,4.0,1.0,59.0,12.457627,1
4,5ca25d74dd12f8002c74364b,Flip The Bird,42.559270,-70.881620,MA,Beverly,BBQ Joint,8.8,22.0,1.0,64.0,12.690000,1


In [460]:
### Check for any NA values 
restaurant_df_outlier_rm[restaurant_df_outlier_rm.isna().any(axis=1) == True ]

,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Average Menu Length,Average Item Price,Division


In [461]:
### Save Cleaned Table to pickle file and test if file was saved properly 
restaurant_df_outlier_rm.to_pickle("restaurant_clean_df_outlier_nan_pickle.pkl")
restaurant_df_outlier_rm = pd.read_pickle("restaurant_clean_df_outlier_nan_pickle.pkl")
restaurant_df_outlier_rm.head()

,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Average Menu Length,Average Item Price,Division
0,534f07e9498e5cc70137182b,The Causeway Restaurant and Pub,42.364659,-71.062912,MA,Boston,BBQ Joint,7.0,55.0,1.0,45.0,9.595556,1
1,593d9fb5c876c8327eef128d,Rusty Can,42.755437,-70.938839,MA,Byfield,BBQ Joint,8.0,14.0,1.0,18.0,8.618889,1
2,4ba2b7caf964a520211338e3,Joff's Backyard Grill,42.084574,-71.471883,MA,Bellingham,BBQ Joint,7.9,18.0,1.0,27.0,8.490000,1
3,5c5efed8419a9e002ce8ea9c,The Smoke Shop BBQ - Assembly Row,42.392249,-71.078180,MA,Somerville,Restaurant,7.7,4.0,1.0,59.0,12.457627,1
4,5ca25d74dd12f8002c74364b,Flip The Bird,42.559270,-70.881620,MA,Beverly,BBQ Joint,8.8,22.0,1.0,64.0,12.690000,1


In [462]:
restaurant_df_outlier_rm["Average Menu Length"].max()

74.0

In [446]:
test =  restaurant_df[ restaurant_df["Name"] == "Daddy D's BBQ" ]
test

,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Menu Item,Menu Item Price
463,5b50aed1c0af57002c6ad928,Daddy D's BBQ,35.605235,-78.802488,NC,Fuquay-Varina,BBQ Joint,-1.0,0,Cheap,[],[]


In [436]:
test = restaurant_df[ restaurant_df["Name"] == "Johnson Family Barbecue" ]["Menu Item Price"]

In [447]:
restaurant_df.head()

,ID,Name,Latitude,Longitude,State,City,Category Name,Rating,Number of Rating,Price Tier,Menu Item,Menu Item Price
0,534f07e9498e5cc70137182b,The Causeway Restaurant and Pub,42.364659,-71.062912,MA,Boston,BBQ Joint,7.0,55,Moderate,[],[]
1,593d9fb5c876c8327eef128d,Rusty Can,42.755437,-70.938839,MA,Byfield,BBQ Joint,8.0,14,Moderate,[],[]
2,4ba2b7caf964a520211338e3,Joff's Backyard Grill,42.084574,-71.471883,MA,Bellingham,BBQ Joint,7.9,18,Moderate,[],[]
3,5c5efed8419a9e002ce8ea9c,The Smoke Shop BBQ - Assembly Row,42.392249,-71.078180,MA,Somerville,Restaurant,-1.0,0,Moderate,"[17Th Street Soul Rolls, Hot Links & Pimento C...","[9.00, 7.50, 9.00, 10.00, 12.00, 9.00, 9.00, 8..."
4,5ca25d74dd12f8002c74364b,Flip The Bird,42.559270,-70.881620,MA,Beverly,BBQ Joint,-1.0,0,Moderate,[],[]


In [437]:
test.values[0]

[{'0': '4.99'},
 {'0': '4.99'},
 {'0': '4.99'},
 {'0': '4.99'},
 {'0': '5.74'},
 {'0': '4.99'},
 {'0': '3.99'},
 {'0': '6.99'},
 {'0': '3.99'},
 {'0': '5.99'},
 {'0': '5.99'},
 {'0': '8.99'},
 {'0': '7.99'},
 {'0': '9.99'},
 {'0': '8.99'},
 {'0': '8.99'},
 {'0': '12.99'},
 {'0': '8.99'},
 {'0': '8.99'},
 {'0': '9.99'},
 {'0': '9.99'},
 {'0': '8.99'},
 {'0': '9.99'},
 {'0': '13.99'},
 {'0': '7.99'},
 {'0': '14.99'},
 {'0': '18.99'},
 {'0': '16.99'},
 {'0': '8.99'},
 {'0': '6.99'},
 {'0': '3.99'},
 {'0': '7.99'},
 {'0': '6.99'},
 {'0': '6.99'},
 {'0': '11.99'},
 {'0': '6.99'},
 {'0': '2.99'},
 {'0': '3.99'},
 {'0': '3.99'},
 {'0': '3.99'},
 {'0': '5.99'},
 {'0': '4.99'},
 {'0': '4.49'},
 {'0': '3.99'},
 {'0': '4.99'},
 {'0': '23.99'},
 {'0': '23.99'},
 {'0': '44.99'},
 {'0': '69.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'},
 {'0': '2.99'}